In [1]:
import pandas as pd
import preprocesing
import wordFeature

In [2]:
def create_feature_dataset(audio_files, preprocessor, feature_extractor):
    all_features = []
    for idx, file_info in enumerate(audio_files):
        # Preprocesar audio
        audio = preprocessor.process_audio(file_info['ruta_archivo'])
        
        # Extraer características
        features = feature_extractor.process_audio(audio)
        
        # Agregar etiquetas
        features['palabra'] = file_info['palabra']
        # features['persona'] = file_info['persona']
        
        all_features.append(features)
        
        if (idx + 1) % 100 == 0:
            print(f"Procesados {idx + 1} archivos de {len(audio_files)}")
            
    return pd.DataFrame(all_features)

In [3]:
# Crear instancias
preprocessor = preprocesing.AudioPreprocessor()
feature_extractor = wordFeature.WordFeatureExtractor()

df = pd.read_csv('dataset.csv')

features_df = create_feature_dataset(df.to_dict('records'), preprocessor, feature_extractor)

features_df.to_csv('voice_features.csv', index=False)

Procesados 100 archivos de 2000
Procesados 200 archivos de 2000
Procesados 300 archivos de 2000
Procesados 400 archivos de 2000
Procesados 500 archivos de 2000
Procesados 600 archivos de 2000
Procesados 700 archivos de 2000
Procesados 800 archivos de 2000
Procesados 900 archivos de 2000
Procesados 1000 archivos de 2000
Procesados 1100 archivos de 2000
Procesados 1200 archivos de 2000
Procesados 1300 archivos de 2000
Procesados 1400 archivos de 2000
Procesados 1500 archivos de 2000
Procesados 1600 archivos de 2000
Procesados 1700 archivos de 2000
Procesados 1800 archivos de 2000
Procesados 1900 archivos de 2000
Procesados 2000 archivos de 2000
